In [1]:
import dask.dataframe as dd
from dask.distributed import Client

Client()

/home/bird/miniconda3/envs/ovscrptd/lib/python3.6/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
/home/bird/miniconda3/envs/ovscrptd/lib/python3.6/site-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


Client Scheduler: tcp://127.0.0.1:36721 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 12 Memory: 33.35 GB


DASK TIP 0: Click on the dashboard link - you'll get lots of helpful progress bars so you can see how your computation is going.


DASK TIP 1: Only read the columns you need - things will go much faster

In [2]:
df = dd.read_parquet(
    'sample_10percent_value_1000_only.parquet', engine='pyarrow',
    columns=['argument_0', 'symbol', 'location', 'script_url', 'value_1000']
)

In [3]:
# Get hs-analytics scripts
hs = df[df.script_url.str.contains('hs-analytics')]
hs.head()

,argument_0,symbol,location,script_url,value_1000
27753,None,window.document.cookie,https://www.accenture.com/ca-en/terms-of-use,https://js.hs-analytics.net/analytics/15133893...,AMCV_AAB73BC75245B44A0A490D4D%40AdobeOrg=14061...
27754,None,window.document.cookie,https://www.accenture.com/ca-en/terms-of-use,https://js.hs-analytics.net/analytics/15133893...,AMCV_AAB73BC75245B44A0A490D4D%40AdobeOrg=14061...
27755,None,window.document.cookie,https://www.accenture.com/ca-en/terms-of-use,https://js.hs-analytics.net/analytics/15133893...,AMCV_AAB73BC75245B44A0A490D4D%40AdobeOrg=14061...
27756,None,window.document.cookie,https://www.accenture.com/ca-en/terms-of-use,https://js.hs-analytics.net/analytics/15133893...,"hsfirstvisit=;expires=Sun, 01 Jan 2006 00:00:0..."
27757,None,window.document.cookie,https://www.accenture.com/ca-en/terms-of-use,https://js.hs-analytics.net/analytics/15133893...,AMCV_AAB73BC75245B44A0A490D4D%40AdobeOrg=14061...


In [4]:
# Get akam scripts
akam = df[df.script_url.str.contains('/akam/')]
akam.head()

,argument_0,symbol,location,script_url,value_1000
6091,None,window.navigator.plugins[Shockwave Flash].length,http://www.elcorteingles.es/moda/accesorios/es...,http://www.elcorteingles.es/akam/10/240f2be0,2
6092,None,window.navigator.plugins[Shockwave Flash].name,http://www.elcorteingles.es/moda/accesorios/es...,http://www.elcorteingles.es/akam/10/240f2be0,Shockwave Flash
6093,None,window.navigator.plugins[Shockwave Flash].desc...,http://www.elcorteingles.es/moda/accesorios/es...,http://www.elcorteingles.es/akam/10/240f2be0,Shockwave Flash 28.0 r0
6094,None,window.navigator.plugins[Shockwave Flash].file...,http://www.elcorteingles.es/moda/accesorios/es...,http://www.elcorteingles.es/akam/10/240f2be0,libflashplayer.so
6095,None,window.navigator.mimeTypes[application/x-shock...,http://www.elcorteingles.es/moda/accesorios/es...,http://www.elcorteingles.es/akam/10/240f2be0,Shockwave Flash


In [5]:
# Get fingerprintjs2 scripts
fp2 = df[df.script_url.str.contains('fingerprint2')]
fp2.head()

,argument_0,symbol,location,script_url,value_1000


In [6]:
len(fp2)

10649

WHAT'S GOING ON?! `head()` returned nothing, but the next cell is saying that the dataframe is 10k rows long.

DASK TIP 2: `head()` returns quickly because it only looks in the first partition.

DASK TIP 3: You need to call `compute()` to get a full set of results or perform certain actions which automatically start computing - most common are len and writing to disk

DASK TIP 4: If you just want to quickly look at some data just get the first partition with `get_partition`

In [7]:
part_0 = df.get_partition(0)
print(f'''
    Complete dataset is {len(df):,} rows.
    Partition is {len(part_0):,} rows.
    Type of dataset is {type(df)}.
    Type of part_0 is {type(part_0)}.
''')


    Complete dataset is 11,292,867 rows.
    Partition is 37,022 rows.
    Type of dataset is <class 'dask.dataframe.core.DataFrame'>.
    Type of part_0 is <class 'dask.dataframe.core.DataFrame'>.



DASK TIP 5: Calling `compute()` returns a pandas dataframe

In [8]:
part_0_pandas = part_0.compute()
print(f'''
    Type of part_0_pandas is {type(part_0_pandas)}.
    part_0_pandas is {len(part_0_pandas):,} rows.
''')


    Type of part_0_pandas is <class 'pandas.core.frame.DataFrame'>.
    part_0_pandas is 37,022 rows.



DASK TIP 6: Can you fit your data in memory? If yes, then you can just use regular pandas. Find out with the memory usage tool. If you've only selected a few small columns you may be good to go.

In [9]:
df = dd.read_parquet(
    'sample_10percent_value_1000_only.parquet', engine='pyarrow',
    columns=['argument_0', 'symbol', 'script_url']
)
memory_usage = df.memory_usage(deep=True).compute()

In [10]:
memory_usage / 1e9  # Print in GB

Index         0.000024
argument_0    0.718110
script_url    1.528499
symbol        0.923305
dtype: float64

In [11]:
memory_usage.sum() / 1e9

3.169938267

3GB - Great - I can just take the whole dataframe and convert it into a pandas dataframe

In [12]:
df = df.compute()

In [13]:
df.head()

,argument_0,symbol,script_url
0,None,window.name,https://vk.com/js/api/xdm.js?1449919642
1,None,window.name,https://vk.com/js/api/xdm.js?1449919642
2,None,window.navigator.userAgent,https://vk.com/js/al/aes_light.js?592436914
3,None,window.navigator.userAgent,https://cpro.baidustatic.com/cpro/ui/noexpire/...
4,None,window.document.cookie,https://apis.google.com/js/plusone.js?_=151338...


Let's finish by looking for all the different ways fingerprintjs2 (or scripts very like it) may be referenced in the dataset.  Instead of just looking for fingerprintjs2 in the name we can look for the characteristic value `Cwm fjordbank glyphs vext quiz`

In [15]:
fp_arguments = df[df.argument_0.str.contains('Cwm fjordbank glyphs vext quiz')]
fp_arguments.head()

,argument_0,symbol,script_url
543,"Cwm fjordbank glyphs vext quiz, 😃",CanvasRenderingContext2D.fillText,https://www.coches.net/ztkieflaaxcvaiwh121837.js
546,"Cwm fjordbank glyphs vext quiz, 😃",CanvasRenderingContext2D.fillText,https://www.coches.net/ztkieflaaxcvaiwh121837.js
5106,"Cwm fjordbank glyphs vext quiz, 😃",CanvasRenderingContext2D.fillText,https://misc.gomein.net.cn/js/regfront/gfp.js
5109,"Cwm fjordbank glyphs vext quiz, 😃",CanvasRenderingContext2D.fillText,https://misc.gomein.net.cn/js/regfront/gfp.js
25438,"Cwm fjordbank glyphs vext quiz, 馃槂",CanvasRenderingContext2D.fillText,https://g.alicdn.com/security/umscript/3.3.2/u...


In [16]:
fp_arguments.script_url.unique()

array(['https://www.coches.net/ztkieflaaxcvaiwh121837.js',
       'https://misc.gomein.net.cn/js/regfront/gfp.js',
       'https://g.alicdn.com/security/umscript/3.3.2/um.js',
       'http://emcharts.dfcfw.com/usercollect/usercollect.min.js',
       'http://pxlclnmdecom-a.akamaihd.net/javascripts/browserfp.min.js?templateId=3',
       'https://m.stripe.network/inner.html#referrer=&title=Delivery%20in%20Orlando%20-%20Postmates%20On-Demand%20Delivery&url=https%3A%2F%2Fpostmates.com%2Fcity%2Forlando&muid=5a651076-0bff-4339-af9d-5fdfc4bdbbfa&sid=c08b8ada-1791-4c10-bd33-1814dca8d4ce&preview=false&',
       'https://m.stripe.network/inner.html#referrer=&title=Calendly%20-%20Scheduling%20appointments%20and%20meetings%20is%20super%20easy%20with%20Calendly.&url=https%3A%2F%2Fcalendly.com%2Fpages%2Fteams&muid=b772201b-e2a8-4fb9-bd16-ad7d1b0aad71&sid=de18442a-70f3-47c5-9273-f7c15f856890&preview=false&',
       'https://img.mediaplex.com/0/711/fingerprint2.js',
       'http://gmyze.com/current/js/